<a href="https://colab.research.google.com/github/mohyunyang/My_Kaggle/blob/main/Porto_Seguro%E2%80%99s_Safe_Driver_Prediction_Step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기

In [ ]:
! pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yhyunmo","key":"8fdb7bbf0193b24d0178986819fcf478"}'}

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle competitions download -c porto-seguro-safe-driver-prediction
!ls
!unzip porto-seguro-safe-driver-prediction.zip

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
 95% 73.0M/76.5M [00:02<00:00, 29.2MB/s]
100% 76.5M/76.5M [00:02<00:00, 27.6MB/s]
kaggle.json  porto-seguro-safe-driver-prediction.zip  sample_data
Archive:  porto-seguro-safe-driver-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd

train = pd.read_csv('train.csv',index_col='id')
test = pd.read_csv('test.csv',index_col='id')
submission = pd.read_csv('sample_submission.csv',index_col='id')

# 2. 피처 엔지니어링

In [ ]:
all_data = pd.concat([train,test], ignore_index = True)
all_data = all_data.drop('target', axis=1)
all_data

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,9,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,13,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,16,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,1488022,0,1,6,0,0,0,1,0,0,...,4,2,3,4,0,1,0,0,1,0
1488024,1488023,5,3,5,1,0,0,0,1,0,...,6,2,2,11,0,0,1,1,0,0
1488025,1488024,0,1,5,0,0,1,0,0,0,...,5,2,2,11,0,1,1,0,0,0
1488026,1488025,6,1,5,1,0,0,0,0,1,...,1,1,2,7,1,1,0,0,0,0


In [ ]:
all_features = all_data.columns # 전체피처
all_features

Index(['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtyp

> 명목형 피처 원핫인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])
encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

> 필요없는 피처제거 

In [ ]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# 1) 명목형 2) calc 분류, 3) 추가제거할 피처를 제외한 피처
remaining_features = [ feature for feature in all_features 
                      if ('cat' not in feature and
                          'calc' not in feature and
                          feature not in drop_features)]

In [ ]:
remaining_features

['id',
 'ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15']

In [ ]:
print(all_data.shape)
print(encoded_cat_matrix.shape)

(1488028, 58)
(1488028, 184)


In [ ]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')
all_data_sprs

<1488028x202 sparse matrix of type '<class 'numpy.float64'>'
	with 39117001 stored elements in Compressed Sparse Row format>

> 데이터 나누기

In [ ]:
num_train = len(train)

# 훈련과 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values # array 형식으로 저장 array([0, 0, 0, ..., 0, 0, 0])

# 3. 평가지표함수작성

본 경진 대회 평가지표는 정규화된 지니계수이다. 이 지표는 파이썬, 사이킷런에서 제공하지 않음. 해서 함수를 만들겠음

`Note : 지니계수 값 = 2 x ROC AUC - 1 `
평가지표가 지니계수이면 ROC AUC 인 상황과 비슷함

In [ ]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]                      # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)       # 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)       # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [ ]:
# LightGBM용 gini 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True # 반환값

  # eval_gini(labels, preds) 평가점수
  # true 평가 점수가 높을수록 좋은지 여부
  # 'gini' 평가지표 이름

# 4. 모델훈련 및 성능검증

> OOF 예측방식

Out of Fold prediction 

K폴드 교차검증을 수행하면서 각 폴드마다 테스트 데이터로 예측하는 방식 (캐글에서 자주 사용하는 예측기법이다)

과대적합방지효과(K폴드방식이기 떄문)


앙상블효과로 모델성능이 좋아진다(K번 예측해 평균한 확률이 한 번만 예측한 확률보다 일반적으로 더 정확)

- 1. 전체 훈련데이터를 K개 그룹으로 나눈다
- 2. K개 그룹 중 한 그룹은 검증데이터, 나머지 K-1 개 그룹은 훈련데이터로 지정함
- 3. 훈련데이터로 모델을 훈련한다.
- 4. 훈련된 모델을 이용해 검증 데이터로 타깃 확률을 예측하고, 전체 테스트 데이터로도 타기 확률을 예측한다.
- 5. 검증데이터로 구한 예측확률과 테스트 데이터로 구한 예측확률을 기록한다.
- 6. 검증데이터를 다른 그룹으로 바꿔가며 2~5 번 절차를 총 K번 반복한다
- 7. K개 그룹의 검증 데이터로 예측한 확률을 훈련데이터 실제 타깃값과 비교해 성능 평가점수를 계산합니다. 이 점수로 모델 성능을 가늠해볼 수 있다.
- 8. 테스트 데이터로 구한 K개 예측 확률의 평균을 구합니다. 이 값이 최종 예측 확률이며 , 제출해야 하는 값이다.

정리하자면 K = 5 라고 했을 때, 총 5개의 모델이 생성이되고 각 모델의 4/5는 훈련데이터로, 나머지 1/5는 검증데이터로 활용되며 훈련과 평가가 진행된다.

5개의 모델이 각자 전체테스트데이터를 사용해 타깃 확률을 예측한다.

그런다음 예측확률 5개의 평균을 구한다. 

이 평균이 최종 예측확률이다.

> OOF 방식으로 LightGBM 훈련

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 층화 K폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991) 

↑↑↑↑시계열 데이터가 아니면 섞어주는게 좋을 수 있다. 

In [ ]:
params = {'objective': 'binary',
          'learning_rate': 0.01,
          'force_row_wise': True,
          'random_state': 0}

In [ ]:
# OFF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])

# OFF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

In [ ]:
print(oof_val_preds)
print(oof_test_preds)
print(oof_val_preds.shape)
print(oof_test_preds.shape)

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
(595212,)
(892816,)


In [ ]:
import lightgbm as lgb

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성 
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋

    # LightGBM 모델 훈련 
    lgb_model = lgb.train(params=params,        # 훈련용 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=1000, # 부스팅 최대반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval=gini,           # 검증용 평가지표
                          early_stopping_rounds=100, # 조기종료 조건
                          verbose_eval=100)     # 100번째마다 점수 출력
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수 
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.153361	valid_0's gini: 0.261414
[200]	valid_0's binary_logloss: 0.152422	valid_0's gini: 0.276386
[300]	valid_0's binary_logloss: 0.152021	valid_0's gini: 0.282761
[400]	valid_0's binary_logloss: 0.151806	valid_0's gini: 0.286927
[500]	valid_0's binary_logloss: 0.151729	valid_0's gini: 0.288247
[600]	valid_0's binary_logloss: 0.151687	valid_0's gini: 0.289139
[700]	valid_0's binary_logloss: 0.151663	valid_0's gini: 0.289936
[800]	valid_0's binary_logloss: 0.151653	valid_0's gini: 0.290213
[900]	valid_0's binary_logloss: 0.15165	valid_0's gini: 0.290355
[1000]	valid_0's binary_logloss: 0.151645	valid_0's gini: 0.290507
Did not meet early stopping. Best iteration is:
[998]	valid_0's binary_logloss: 0.151644	valid_0's gini: 0.290524
폴드 1 지니계수 : 0.2905237882587803

#####################################

In [ ]:
print(oof_val_preds[:5])
print(oof_test_preds[:5])

[0.07721805 0.03143718 0.02000396 0.01725605 0.04090653]
[0.03946195 0.03351071 0.03871636 0.02300037 0.04515248]


array([0.02149865, 0.0234405 , 0.02959405, ..., 0.05914199, 0.04272677,
       0.01880705])

In [ ]:
print('OOF 검증 데이터 지니계수:', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수: 0.28021261417854254


# 5. 예측 및 결과 제출

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission_1.csv')